![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# PEC 2: Parte 2, Marzo 2023

## Extracción de conocimiento de fuentes de datos heterogéneas mediante Spark SQL, RDDs y GraphFrames

En esta práctica vamos a introducir estructuras de datos más complejas que las vistas hasta ahora, donde los campos pueden a su vez tener campos anidados. En concreto utilizaremos datos de twitter capturados en el contexto de las elecciones generales en España del 28 de Abril de 2019. La práctica está estructurada de la siguiente manera:
- **Parte 0:** Configuración del entorno
- **Parte 1:** Introducción a data frames estructurados y cómo operar extraer información *(3 puntos)*
    - **Parte 1.1:** Importar los datos *(1 puntos)*
    - **Parte 1.2:** *Queries* sobre sobre data frames complejos *(2 puntos)*
        - **Parte 1.2.1:** Queries SQL *(1 puntos)*
        - **Parte 1.2.2:** Queries sobre el pipeline *(1 puntos)*
- **Parte 2:** Bases de datos HIVE y operaciones complejas *(3 puntos)*
    - **Parte 2.1:** Bases de datos Hive *(1 puntos)*
    - **Parte 2.2:** Más allá de las transformaciones SQL *(2 puntos)*
        - **Parte 2.2.1:** Tweets por población  *(1 puntos)*
        - **Parte 2.2.2:** Contar hashtags *(1 puntos)*
- **Parte 3:** Sampling *(2 Puntos)*
- **Parte 4**: Introducción a los datos relacionales *(2 puntos)*
     - **Parte 4.1**: Construcción de la edgelist *(1 puntos)*
     - **Parte 4.2**: Centralidad de grado *(1 puntos)*


## **Parte 0:** Configuración del entorno

In [ ]:
import findspark
findspark.init()

In [ ]:
import re
import os
import pandas as pd
from matplotlib import pyplot as plt
from math import floor
from pyspark import SparkConf, SparkContext, SQLContext, HiveContext
from pyspark.sql import Row

In [ ]:
SUBMIT_ARGS = "--packages graphframes:graphframes:0.7.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

conf = SparkConf()
conf.setMaster("local[1]")
# Introducid el nombre de la app PEC3_ seguido de vuestro nombre de usuario
conf.setAppName(<FILL IN>)
sc = SparkContext(conf=conf)

## **Parte 1:** Introducción a data frames estructurados y operaciones sobre ellos.

Como ya se ha mencionado, en esta práctica vamos ha utilizar datos de Twitter que recolectamos durante las elecciones generales en España del 28 de abril de 2019. Como veremos, los tweets tienen una estructura interna bastante compleja que hemos simplificado un poco en esta práctica.

### **Parte 1.1:** Importar los datos

Lo primero que vamos ha aprender es cómo importar este tipo de datos a nuestro entorno. Uno de los tipos de archivos más comunes para guardar este formato de información es [la estructura JSON](https://en.wikipedia.org/wiki/JSON). Esta estructura permite guardar información en un texto plano de diferentes objetos siguiendo una estructura de diccionario donde cada campo tiene asignado una llave y un valor. La estructura puede ser anidada, o sea que una llave puede tener como valor otra estructura tipo diccionario.

Spark SQL permite leer datos de muchos formatos diferentes (como recordareis de la anterior práctica donde leímos un fichero CSV). En esta ocasión, se os pide que leáis un fichero JSON de la ruta ```/aula_22.419/data/tweets28a_sample.json```. Este archivo contiene un pequeño *sample*, un 0.1% de la base de datos completa (en un siguiente apartado veremos cómo realizar este *sampleado*). En esta ocasión no se os pide especificar la estructura del data frame ya que la función de lectura la inferirá automáticamente.


In [ ]:
sqlContext = SQLContext(sc)
tweets_sample = sqlContext.read.json(<FILL IN>)

print("Loaded dataset contains %d tweets" % tweets_sample.count())

El siguiente paso es mostrar la estructura del dataset que acabamos de cargar. Recordad que podéis obtener la información acerca de cómo está estructurado el DataTable utilizando el método ```printSchema()```.

In [ ]:
tweets_sample.<FILL IN>

Podéis observar que la estructura del tweet contiene múltiples campos anidados. Teneis que familiarizaros con esta estructura ya que será la que utilizaremos durante toda la práctica. Recordad también que no todos los tweets tienen todos los campos, como por ejemplo la ubicación (campo ```place```). Cuando esto pasa el campo pasa a ser ```NULL```. Podéis ver mas información sobre este tipo de datos en [este enlace](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object).

### **Parte 1.2:** *Queries* sobre sobre data frames complejos

En esta parte vamos a introducir conceptos sobre cómo trabajar con data tables.

#### **Parte 1.2.1:** Queries SQL

El primer paso consiste en registrar la tabla en el contexto SQL comprobando primero si existe y borrándola en el caso que sea así. En este apartado se os pide que registréis la tabla ```tweets_sample``` que acabamos de cargar en el contexto sql bajo el mismo nombre ```tweets_sample```.

In [ ]:
sqlContext.sql(<FILL IN>)
sqlContext.registerDataFrameAsTable(<FILL IN>)

Ahora se os pide que creeis una tabla ```users_agg``` con [la información agregada](https://www.w3schools.com/sql/sql_groupby.asp) de los usuarios que tengan definido su idioma (```user.lang```) como español (```es```). En concreto se os pide que la tabla contenga las siguientes columnas:
- **screen_name:** nombre del usuario
- **friends_count:** número máximo (ver nota) de personas a las que sigue
- **tweets:** número de tweets realizados
- **followers_count:** número máximo (ver nota) personas que siguen al usuario.

El orden en el cual se deben mostrar los registros es orden descendente acorde al número de tweets.

***Nota:*** es importante que os fijéis que el nombre de *friends* i *followers* puede diferir a lo largo de la adquisición de datos. En este caso vamos ha utilizar la función de agregación ```MAX``` sobre cada uno de estos campos para evitar segmentar el usuario en diversas instancias.

In [ ]:
users_agg = sqlContext.sql("""  SELECT <FILL_IN>, MAX(user.friends_count) AS friends_count, <FILL_IN>, MAX(user.followers_count) AS followers_count
                                FROM tweets_sample
                                WHERE user.lang == 'es'
                                GROUP BY <FILL_IN>
                                ORDER BY <FILL_IN>""")
users_agg.limit(10).show()

In [ ]:
output = users_agg.first()
assert output.screen_name == 'anaoromi' and output.friends_count == 6258 and output.tweets == 16 and output.followers_count == 6774, "Incorrect output"

Imaginad ahora que queremos combinar la información que acabamos de generar con información acerca del número de veces que un usuario ha sido retuiteado. Para hacer este tipo de combinaciones necesitamos recurrir al [```JOIN``` de tablas](https://www.w3schools.com/sql/sql_join.asp). Primero debemos registrar la tabla que acabamos de generar en el contexto SQL. Recordad que primero debéis comprobar si la tabla existe y en caso afirmativo eliminarla ('drop table if exists <tabla>'). La tabla tenéis que registrarla bajo el nombre de ```user_agg```.

In [ ]:
sqlContext.sql(<FILL IN>)
sqlContext.registerDataFrameAsTable(<FILL IN>)

Una vez registrada se pide que combinéis esta tabla y la tabla ```tweets_sample``` utilizando un ```INNER JOIN``` para obtener una nueva tabla con la siguiente información:
- ***screen_name:*** nombre de usuario
- ***friends_count:*** número máximo de personas a las que sigue
- ***followers_count:*** número máximo de personas que siguen al usuario.
- ***tweets:*** número de tweets realizados por el usuario.
- ***retweeted:*** número de retweets obtenidos por el usuario.
- ***ratio_tweet_retweeted:*** ratio de retweets por número de tweets publicados $\frac{retweets}{tweets}$

In [ ]:
retweeted = sqlContext.sql("""  SELECT <FILL IN>, COUNT(*) / user_agg.tweets AS ratio_tweet_retweeted
                                FROM user_agg
                                INNER JOIN tweets_sample
                                WHERE <FILL IN>
                                GROUP BY user_agg.screen_name, user_agg.friends_count, user_agg.followers_count, user_agg.tweets
                                ORDER BY <FILL IN> DESC""")

retweeted.limit(10).show()

In [ ]:
output = retweeted.first()
assert output.screen_name == 'PSOE' and output.friends_count == 13635 and output.tweets == 1 and output.followers_count == 671073 and output.ratio_tweet_retweeted == 155.0 and output.retweeted == 155, "Incorrect output"

#### **Parte 1.2.2:** Queries a través de la API

Las tablas de Spark SQL ofrecen otro mecanismo para aplicar las transformaciones y obtener resultados similares a los que se obtendría aplicando una consulta SQL. Por ejemplo utilizando el siguiente pipeline obtendremos el texto de todos los tweets en español:

```
tweets_sample.where("lang == 'es'").select("text")
```

Que es equivalente a la siguiente sentencia SQL:

```
SELECT text
FROM tweets_sample
WHERE lang == 'es'
```

Podéis consultar el [API de spark SQL](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html) para encontrar más información sobre como utilitzar las diferentes transformaciones en tablas.

En este ejercicio se os pide que repliquéis la query obtenida en el apartado anterior empezando por generar la tabla ```users_agg```. Podéis utilizar las transformaciones ```where```, ```select``` (o ```selectExpr```), ```groupBy```, ```count```, ```agg``` y ```orderBy```

In [ ]:
users = tweets_sample.where("user.lang == 'es'").select(<FILL IN>)

users_agg = users.groupBy(<FILL IN>)\
                 .agg({"screen_name":"count", "followers_count":"max", "friends_count":"max"})\
                 .orderBy("count(screen_name)", ascending=False)

users_agg.limit(10).show()

Si os fijáis veréis que el nombre de las columnas no corresponde con el obtenido anteriormente, podéis cambiar el nombre de una columna determinada utilizando la transformación ```withColumnRenamed```. Cambiad el nombre de las columnas para que coincidan con el apartado anterior y guardadlas en una variable ```user_agg_new```.

In [ ]:
users_agg_new = users_agg.withColumnRenamed("max(friends_count)", "friends_count")\
                         .withColumnRenamed(<FILL IN>)\
                         .withColumnRenamed(<FILL IN>)

users_agg_new.limit(10).show()

In [ ]:
output = users_agg_new.first()
assert output.screen_name == 'anaoromi' and output.friends_count == 6258 and output.tweets == 16 and output.followers_count == 6774, "Incorrect output"

Cread ahora una tabla ```user_retweets``` utilizando transformaciones que contenga dos columnas:
- ***screen_name:*** nombre de usuario
- ***retweeted:*** número de retweets

Podéis utilizar las mismas transformaciones que en el ejercicio anterior. Ordenad la tabla en orden descendente utilizando el valor de la columna ```retweeted```.

In [ ]:
user_retweets = tweets_sample.select(<FILL IN>)\
                             .where("retweeted_status.user.lang == 'es'")\
                             .groupBy(<FILL IN>)\
                             .count()\
                             .withColumnRenamed(<FILL IN>)\
                             .orderBy(<FILL IN>)

user_retweets.limit(10).show()

In [ ]:
output = user_retweets.first()
assert output.screen_name == 'vox_es' and output.retweeted == 299, "Incorrect output"

Otra manera de combinar dos tablas es utilizando el [metodo de tabla ```join```](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html). Combinad la información de la tabla ```users_agg_new``` y ```user_retweets``` en una nueva tabla ```retweeted``` utilizando la columna ```screen_name```. Ordenad la nueva tabla en orden descendente con el nombre de retweets.

In [ ]:
retweeted = users_agg_new.join(<FILL IN>)\
                         .orderBy("retweeted", ascending = False)

retweeted.limit(10).show()

In [ ]:
output = retweeted.first()
assert output.screen_name == 'PSOE' and output.friends_count == 13635 and output.tweets == 1 and output.followers_count == 671073 and output.retweeted == 155, "Incorrect output"

Notaréis que algunos de los registros que aparecen en la tabla ```users_retweeted``` no están presentes en la tabla retweeted. Esto es debido a que, por defecto, el método aplica un inner join y por tanto solo combina los registros presentes en ambas tablas. Podéis cambiar este comportamiento a través de los parámetros de la función.

Para terminar esta parte y reconstruir el resultado del apartado 1.2.1 vamos a añadir una columna ```ratio_tweet_retweeted``` con información del ratio entre retweets y tweets. Para ello debéis utilizar la transformación ```withColumn```. El resultado debe estar ordenado considerando esta nueva columna en orden descendente.

In [ ]:
retweeted = retweeted.withColumn(<FILL_IN>, retweeted[<FILL_IN_NOMBRE_COL>] / retweeted[<FILL_IN_NOMBRE_COL>]).orderBy(<FILL_IN>)
retweeted.limit(10).show()

In [ ]:
output = retweeted.first()
assert output.screen_name == 'PSOE' and output.friends_count == 13635 and output.tweets == 1 and output.followers_count == 671073 and output.ratio_tweet_retweeted == 155.0 and output.retweeted == 155, "Incorrect output"

## **Parte 2:** Bases de datos HIVE y operaciones complejas

Hasta ahora hemos estado trabajando con un pequeño sample de los tweets generados (el 0.1%). En esta parte de la PEC vamos a ver como trabajar y tratar con el dataset completo. Para ello vamos ha utilizar tanto transformaciones sobre tablas como operaciones sobre RDD cuando sea necesario.

### **Parte 2.1:** Bases de datos Hive

Muchas veces los datos con los que vamos ha trabajar se van a utilizar en diversos proyectos. Una manera de organizar los datos es, en lugar de utilizar directamente los ficheros, recurrir a una base de datos para gestionar la información. En el entorno Hadoop una de las bases de datos más utilizadas es [Apache Hive](https://hive.apache.org/), una base de datos que permite trabajar con contenido distribuido.

La manera de acceder a esta base de datos es creando un contexto Hive de manera muy similar a como declaramos un contexto SQL. Primero de todo vamos a declarar un variable ```hiveContext``` instanciándola como un objeto de la classe ```HiveContext```. Acto seguido vamos a comprobar cuantas tablas están registradas en este contexto.

In [ ]:
hiveContext = HiveContext(sc)
hiveContext.tables().show()

Observad que ahora mismo tenemos cinco tablas registradas en este contexto. Tres de ellas no temporales y dos temporales, las que hemos registrado previamente. Por tanto sqlContext y hiveContext están concetados (es la misma sessión)

Vamos ha crear una variable ```tweets``` que utilizaremos para acceder a la tabla ```tweets28a``` guardada en ```hiveContext``` utilizando para ello el método ```table()``` de este objeto.

In [ ]:
tweets = hiveContext.table(<FILL_IN>)
print("Loaded dataset contains {} tweets".format(tweets.count()))

Utilizando el mismo método que en el apartado 1.1, comprobad la estructura de la tabla que acabamos de cargar con ```printSchema()```

In [ ]:
<FILL IN>

### **Parte 2.2:** Más allá de las transformaciones SQL

Algunas veces vamos a necesitar obtener resultados que precisan operaciones que van más allá de lo que podemos conseguir utilizando el lenguaje SQL. En esta parte de la práctica vamos practicar cómo pasar de una tabla a un RDD, para hacer operaciones complejas, y luego volver a pasar a una tabla.

#### **Parte 2.2.1:** Tweets por población

Un pequeño porcentaje, alrededor del 1%, de los tweets realizados está geolocalizado. Eso quiere decir que para estos tweets tenemos información acerca del lugar donde han sido realizados guardado en el campo ```place```. En este ejercicio se pide que utilizando una sentencia SQL mostréis en orden descendente cuántos tweets se han realizado en cada lugar. La tabla resultante ```tweets_place``` debe tener las siguientes columnas:
- ***name:*** nombre del lugar
- ***tweets:*** número de tweets

Recordad que no todos los tweets en la base de datos tienen que tener información geolocalizada, tenéis que filtrarlos teniendo en cuenta todos los que tienen un valor no nulo.

In [ ]:
tweets_place = hiveContext.sql("""  SELECT place.name, <FILL_IN>
                                    FROM tweets28a
                                    WHERE place IS NOT NULL
                                    GROUP BY <FILL_IN>
                                    ORDER BY <FILL_IN>""")
tweets_place.limit(10).show()

In [ ]:
output = tweets_place.first()
assert output.name == "Madrid" and output.tweets == 19655, "Incorrect output"

#### **Parte 2.2.2:** Contar hashtags

Como seguro que habéis observado la semántica de la sentencia SQL es mucho más limpia que trabajar con RDDs para realizar muchas tareas, pero no todas las que os vais a encontrar se pueden hacer mediante sentencias SQL. En este ejercicio vamos a ver un ejemplo.

El objetivo de este ejercicio es contar el número de veces que cada hashtag (palabras precedidas por un #) ha aparecido en el dataset. Para evitar la sobrerrepresentación debida a los retweets vamos a concentrarnos en solo aquellos tweets que no son retweets de ningún otro, o dicho de otra manera, en aquellos en los que el campo ```retweeted_status``` es nulo. Cread una variable ```non_retweets``` que contenga todos estos tweets.

In [ ]:
non_retweets = tweets.where("retweeted_status IS NULL")

Seguidamente vamos ha crear una variable ```hashtags``` que contenga una lista de tuplas con la información ```(hashtag, count)```. Para ello, cread un RDD que contenga una lista con el texto de todos los tweets. Una vez hecho este paso tenéis que extraer los hashtags (palabras precedidas por un #) y contarlos.

Recordad los conocimientos adquiridos en la PEC 'word count' y el anterior ejercicio, os serán de gran ayuda.

In [ ]:
# Puede modificarse si se considera conveniente el map y el flatMap
hashtags = non_retweets.select("text").rdd.map(<FILL IN>)
                                          .flatMap(lambda line: re.findall(r"#[\w']+", line)

hashtags = hashtags.map(<FILL IN>).reduceByKey(<FILL IN>)

Finalmente, se os pide que con el RDD obtenido generéis una tabla ```hashtagsTable``` compuesta de dos columnas:
- ***hashtag***
- ***num:*** número de veces que aparece cada hashtag.

Ordenadla en orden descendente por número de tweets.

In [ ]:
hashtagsTable = hashtags.map(lambda row: Row(hashtag=row[0], num=row[1]))\
                        .toDF()\
                        .orderBy(<FILL IN>)

hashtagsTable.limit(20).show()

In [ ]:
output = hashtagsTable.first()
assert output.hashtag == "#28A" and output.num == 158124, "Incorrect output"

## **Parte 3:** Sampling

En muchas ocasiones, antes de lanzar costoso procesos, es una práctica habitual tratar con un pequeño conjunto de los datos para investigar algunas propiedades o simplemente para debugar nuestros algoritmos, a esta tarea se la llama sampling.

El sampling aplicado es [el homogeneo](https://en.wikipedia.org/wiki/Simple_random_sample). Este sampling se basta en simplemente escoger una fracción de la población seleccionando aleatoriamente elementos de la misma.

Primero de todo vamos ha realizar un sampling homogéneo del 1% de los tweets generados en periodo electoral sin reemplazo. Guardad en una variable ```tweets_sample``` este sampling utilizando el método ```sample``` descrito en la [API de pyspark SQL](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html). El seed que vais a utilizar para inicializar el generador aleatorio es 42.

In [ ]:
seed = 42
fraction = 0.01

tweets_sample = tweets.sample(False, <FILL IN>)

print("Number of tweets sampled: {0}".format(tweets_sample.count()))

In [ ]:
assert tweets_sample.count() == 254364, "Incorrect output"

Una de las cosas que resulta interesante comprobar acerca de los patrones de uso de las redes sociales es el patrón de uso diario. En este caso nos interesa el número promedio de tweets que se genera cada hora del día. Para extraer esta información lo que haremos primero, será generar una tabla ```tweets_timestamp``` con la información:
- ***created_at***: timestamp de cuando se publicó el tweet.
- ***hour***: a que hora del dia corresponde.
- ***day***: Fecha en formato MM-dd-YY

La fecha que figura en la base de datos esta en la franja horaria GMT. El primer paso es pasar esta información al horario peninsular de España, podéis utilizar la función ```from_utc_timestamp``` para este fin. Así mismo, la función ```hour``` os servirá para extraer la hora del timestamp y la función ```date_format``` os permitirá generar la fecha.

In [ ]:
from pyspark.sql.functions import date_format, hour, from_utc_timestamp

tweets_timestamp = tweets_sample.select("created_at")
tweets_timestamp = tweets_timestamp.withColumn("created_at", from_utc_timestamp(<FILL_IN>))
tweets_timestamp = tweets_timestamp.withColumn("hour",  <FILL_IN>)\
                                   .withColumn(<FILL_IN>)
tweets_timestamp.limit(20).show()

El paso siguiente es agregar estos datos por hora y día en una tabla ```tweets_hour_day```. Tenéis que crear una tabla ```tweets_hour``` con la información:
- ***hour:*** hora del dia
- ***day:*** fecha
- ***count:*** número de tweets generados

In [ ]:
tweets_hour_day = tweets_timestamp.groupBy(<FILL_IN>,<FILL_IN>).count()

tweets_hour_day.limit(20).show()

Por último solo nos queda hacer una agregación por hora para conseguir el promedio de tweets por hora. Tenéis que generar una tabla ```tweets_hour``` con la información:
- ***hour:*** Hora
- ***tweets:*** Promedio de tweets realizados

Recordad que estamos trabajando con un sample del 1% por tanto tenéis que corregir la columna ```tweets``` para que refleje el promedio que deberíamos esperar en el conjunto completo de tweets. La tabla tiene que estar ordenada en orden ascendente de hora.

In [ ]:
tweets_hour = tweets_hour_day.groupBy(<FILL_IN>)\
                             .avg(<FILL_IN>)\
                             .orderBy(<FILL_IN>)\
                             .withColumnRenamed(<FILL_IN>)
            
tweets_hour = tweets_hour.withColumn("tweets", tweets_hour.tweets / fraction)
tweets_hour.limit(24).show()


Por último, tenéis que producir un gráfico de barras utilizando Pandas (pasamos el dataframe a Pandas) donde se muestre la información que acabáis de generar.

In [ ]:
tweets_hour_pd = tweets_hour.toPandas()
tweets_hour_pd.plot.bar(<FILL_IN>)

## **Parte 4:** Introducción a los datos relacionales

El hecho de trabajar con una base de datos que contiene información generada en una red social nos permite introducir el concepto de datos relacionales. Podemos definir datos relacionales como aquellos en los que existen relaciones entre las entidades que constituyen la base de datos. Si estas relaciones son binarias, relaciones 1 a 1, podemos representar las relaciones como un grafo compuesto por un conjunto de vértices $\mathcal{V}$ y un conjunto de aristas $\mathcal{E}$ que los relacionan.

En el caso de grafos que emergen de manera orgánica, este tipo de estructura va más allá de los grafos regulares que seguramente conocéis. Este tipo de estructuras se conocen como [redes complejas](https://es.wikipedia.org/wiki/Red_compleja). El estudio de la estructura y dinámicas de este tipo de redes ha contribuido a importantes resultados en campos tan dispares como la física, la sociología, la ecología o la medicina.

![complex_network](https://images.squarespace-cdn.com/content/5150aec6e4b0e340ec52710a/1364574727391-XVOFAB9P6GHKTDAH6QTA/lastfm_800_graph_white.png?content-type=image%2Fpng)

En esta última parte de la práctica vamos ha trabajar con este tipo de datos. En concreto vamos a modelar uno de los posibles relaciones presentes en el dataset, la red de retweets.

### Generar la red de retweets

#### **Parte 4.1**: Construcción de la edgelist

Lo primero se os pide es que generéis la red. Hay diversas maneras de representar una red compleja, por ejemplo, si estuvierais interesados en trabajar en ellas desde el punto de vista teórico, la manera más habitual de representarlas es utilizando una [matriz de adyacencia](https://es.wikipedia.org/wiki/Matriz_de_adyacencia). En esta práctica vamos a centrarnos en el aspecto computacional, una de las maneras de mas eficientes (computacionalmente hablando) de representar una red es mediante su [*edge list*](https://en.wikipedia.org/wiki/Edge_list), una tabla que especifica la relación a parejas entre las entidades.

Las relaciones pueden ser bidireccionales o direccionales y tener algún peso asignado o no (weighted or unweighted). En el caso que nos ocupa, estamos hablando de una red dirigida, un usuario retuitea a otro, y podemos pensarla teniendo en cuenta cuántas veces esto ha pasado.

Lo primero que haréis para simplificar el cómputo,  es crear un sample homogéneo sin reemplazo del 1% de los tweets. Utilizando los conocimientos que habéis aprendido en el apartado 3.1. Utilizaremos 42 como valor para la seed.

In [ ]:
sample = tweets.sample(False, <FILL_IN>).persist()

Ahora vais a crear una tabla ```edgelist``` con la siguiente información:
- ***src:*** usuario que retuitea
- ***dst:*** usuario que es retuiteado
- ***weight:*** número de veces que un usuario retuitea a otro.

Filtrar el resultado para que contenga sólo las relaciones con un weight igual o mayor a dos.

In [ ]:
edgelist = sample.where(<FILL_IN>)\
                 .selectExpr("user.screen_name AS src", "retweeted_status.user.screen_name AS dst")

edgelist = edgelist.groupBy(<FILL_IN>).count().withColumnRenamed("count", "weight").where("weight >= <FILL_IN>")

L = edgelist.count()

print("There are {0} edges on the network.".format(L))

In [ ]:
assert L == 5247, "Incorrect ouput"

#### **Parte 4.2:** Centralidad de grado

Uno de los descriptores más comunes en el análisis de redes es el grado. El grado cuantifica cuántas aristas están conectadas a cada vértices. En el caso de redes dirigidas como la que acabamos de crear este descriptor está descompuesto en el:
- **in degree**: cuantas aristas apuntan al nodo
- **out degree**: cuantas aristas salen del nodo

Si haces un ranquing de estos valores vais a obtener medida de centralidad, la [centralidad de grado](https://en.wikipedia.org/wiki/Centrality#Degree_centrality), de cada uno de los nodos.

Se os pide que generéis una tabla con la información:
- ***screen_name:*** nombre del usuario (anteriormene src).
- ***outDegree:*** out degree del nodo.

Ordenado la tabla por out degree en orden descendente.

In [ ]:
outDegree = edgelist.groupBy("src")\
                    .count()\
                    .orderBy(<FILL_IN>)\
                    .withColumnRenamed("count", <FILL_IN>)\
                    .withColumnRenamed("src", <FILL_IN>)
outDegree.limit(20).show()

In [ ]:
output = outDegree.first()
assert output.screen_name == "rosavergar23" and output.outDegree == 11, "Incorrect output"

Se os pide ahora que generéis una tabla con la información:
- ***screen_name:*** nombre del usuario (anteriormente dst).
- ***inDegree:*** in degree del nodo.

Ordenad la tabla por in degree en orden descendente.

In [ ]:
# Replicando el código utilizado para generar el outDegree, generad el inDegree
inDegree = edgelist.<FILL IN>

inDegree.limit(20).show()

In [ ]:
output = inDegree.first()
assert output.screen_name == "vox_es" and output.inDegree == 330, "Incorrect output"